# Install Dependencies

## External Dependencies

In [9]:
!pip install inflect


[notice] A new release of pip available: 22.2.1 -> 22.2.2
[notice] To update, run: pip install --upgrade pip


## FrostAura Dependencies

In [10]:
import os
import sys

USE_LOCAL_CODE: bool = True # NOTE: When changing this configuration, be sure to restart the interpreter in order to have sys path updates reflect.
directories_to_move_up: int = 2
current_executing_path: str = os.getcwd()
current_executing_path = current_executing_path.replace('\\', '/')
root_path: str = '/'.join(current_executing_path.split('/')[:-directories_to_move_up])

# Clean up.
%pip uninstall frostaura -y
sys.path = [p for p in sys.path if p != root_path]

if USE_LOCAL_CODE:
    sys.path.append(root_path)
else:
    %pip install -U --no-cache-dir frostaura

Note: you may need to restart the kernel to use updated packages.


# Import Dependencies

## Logging

In [11]:
import logging

logging.getLogger().disabled = True

## Imports

In [12]:
from frostaura.engines import IAssetProjectionEngine
from frostaura.engines import SimpleAssetProjectionEngine
import pandas as pd
from frostaura.engines import IAssetValuationEngine
from frostaura.engines import GrahamValuationEngine
from frostaura.engines import IAssetCalculationsEngine
from frostaura.engines import SimpleAssetCalculationsEngine
from frostaura.data_access import EasyEquitiesPersonalAssetDataAccess
from frostaura.data_access import IResourcesDataAccess
from frostaura.data_access import EmbeddedResourcesDataAccess
from frostaura.data_access import HtmlResourcesDataAccess
from frostaura.data_access import IPersonalAssetDataAccess
from frostaura.data_access import IPublicAssetDataAccess
from frostaura.data_access import YahooFinanceDataAccess
from frostaura.models import ValuationResult

# Usage

## Setup

In [13]:
html_data_access: IResourcesDataAccess = HtmlResourcesDataAccess()
resource_data_access: IResourcesDataAccess = EmbeddedResourcesDataAccess()
public_asset_data_access: IPublicAssetDataAccess = YahooFinanceDataAccess()
asset_valuation_engine: IAssetValuationEngine = GrahamValuationEngine(html_data_access=html_data_access, public_asset_data_access=public_asset_data_access)
asset_calculations_engine: IAssetCalculationsEngine = SimpleAssetCalculationsEngine(public_asset_data_access=public_asset_data_access)
personal_asset_data_access: IPersonalAssetDataAccess = EasyEquitiesPersonalAssetDataAccess(resource_data_access=resource_data_access,
                                                                         username=None,
                                                                         password=None)
instance: IAssetProjectionEngine = SimpleAssetProjectionEngine(asset_valuation_engine=asset_valuation_engine,
                                                               public_asset_data_access=public_asset_data_access)

## Project for a Single Symbol

In [14]:
n_months: int = 12*30 # 39 year
usd_zar_exchange_rate: float = 16.17
symbol: str = 'AAPL'
holdings: pd.DataFrame = personal_asset_data_access.get_personal_transactions()
holdings_with_profits: pd.DataFrame = asset_calculations_engine.calculate_holdings_profits(holdings=holdings)
initial_value: float = holdings_with_profits.loc[holdings_with_profits['symbol'] == 'AAPL']['total_current_usd'].sum()
monthly_deposit: float = 2000 / usd_zar_exchange_rate / len(holdings_with_profits)

growth: pd.DataFrame = instance.project_monthly_asset_growth(n_months=n_months,
                                                             symbol=symbol,
                                                             principal_value=initial_value,
                                                             monthly_deposit=monthly_deposit)

growth

,deposits_withdrawals,interest,total_deposits_withdrawals,accrued_interest,balance
month,,,,,
0,0.000000,0.000000,0.000000,0.000000,70.634752
1,24.737168,0.583325,24.737168,0.583325,95.955245
2,24.737168,0.792430,49.474335,1.375756,121.484843
3,24.737168,1.003262,74.211503,2.379018,147.225273
4,24.737168,1.215835,98.948670,3.594853,173.178276
...,...,...,...,...,...
356,24.737168,444.582027,8806.431664,45426.605234,54303.671650
357,24.737168,448.457822,8831.168831,45875.063056,54776.866639
358,24.737168,452.365624,8855.905999,46327.428680,55253.969430


## Project for a Entire Holdings (Multiple Symbols)

In [15]:
# 30 years
n_months: int = 12*30
usd_zar_exchange_rate: float = 16.17
holdings: pd.DataFrame = personal_asset_data_access.get_personal_transactions()
holdings_with_profits: pd.DataFrame = asset_calculations_engine.calculate_holdings_profits(holdings=holdings)
# Simply R 1000 converted to USD and split evenly among how many assets are held.
monthly_deposits: list = [(2000 / usd_zar_exchange_rate / len(holdings_with_profits)) for h in holdings_with_profits]
holdings_growth = instance.project_monthly_holdings_growth(n_months=n_months,
                                                           holdings_with_profits=holdings_with_profits,
                                                           monthly_deposits=monthly_deposits)

holdings_growth

,month,deposits_withdrawals,interest,total_deposits_withdrawals,accrued_interest,balance
0,1,98.94867,0.017060,98.948670,5.856276e+00,4.422146e+02
1,2,98.94867,0.017408,197.897341,1.369068e+01,5.489976e+02
2,3,98.94867,0.017659,296.846011,2.355950e+01,6.578151e+02
3,4,98.94867,0.017859,395.794682,3.552086e+01,7.687252e+02
4,5,98.94867,0.018029,494.743352,4.963478e+01,8.817878e+02
...,...,...,...,...,...,...
355,356,98.94867,0.036041,35225.726654,3.599549e+08,3.599905e+08
356,357,98.94867,0.036045,35324.675325,3.734158e+08,3.734514e+08
357,358,98.94867,0.036048,35423.623995,3.873813e+08,3.874170e+08
358,359,98.94867,0.036051,35522.572665,4.018703e+08,4.019061e+08


## Project Wealth Over Months

In [16]:
monthly_zar_deposit: float = 2000
usd_zar_exchange_rate: float = 16.17
monthly_deposits: list = [(monthly_zar_deposit / usd_zar_exchange_rate / len(holdings_with_profits)) for h in holdings_with_profits]
wealth_projection = instance.project_wealth_growth(holdings_with_profits=holdings_with_profits,
                                                   monthly_deposits=monthly_deposits)

wealth_projection

,Target Balance (USD),Target Balance Eng (USD),ETA (Months),ETA (Years)
0,1.114621e+03,"one thousand, one hundred and fourteen point s...",6,0.500000
1,1.003311e+04,ten thousand and thirty-three point one one ze...,50,4.166667
2,1.020855e+05,one hundred and two thousand and eighty-five p...,124,10.333333
3,1.003542e+06,"one million, three thousand, five hundred and ...",192,16.000000
4,1.011175e+07,"ten million, one hundred and eleven thousand, ...",257,21.416667
5,1.035632e+08,"one hundred and three million, five hundred an...",321,26.750000
6,1.007346e+09,"one billion, seven million, three hundred and ...",383,31.916667
7,1.026224e+10,"ten billion, two hundred and sixty-two million...",446,37.166667
